In [4]:
import tkinter as tk
from tkinter import ttk
import heapq

class LinearProgramSolver:
    def __init__(self, root):
        self.root = root
        self.root.title("Linear Program Solver")

        self.create_input_widgets()
        self.create_output_widgets()


    def create_input_widgets(self):
        self.input_frame = tk.LabelFrame(self.root, text="Input", padx=10, pady=10)
        self.input_frame.grid(row=0, column=0, padx=5, pady=5, sticky="we")

        self.num_variables_label = tk.Label(self.input_frame, text="Number of variables:")
        self.num_variables_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.num_variables_entry = tk.Entry(self.input_frame)
        self.num_variables_entry.grid(row=0, column=1, padx=5, pady=5)

        self.num_constraints_label = tk.Label(self.input_frame, text="Number of constraints:")
        self.num_constraints_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.num_constraints_entry = tk.Entry(self.input_frame)
        self.num_constraints_entry.grid(row=1, column=1, padx=5, pady=5)

        self.update_button = tk.Button(self.input_frame, text="Update", command=self.update_variables_constraints)
        self.update_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

        self.input_variables = []
        self.input_constraints = []
        self.input_b_values = []

    def create_output_widgets(self):
        self.output_frame = tk.LabelFrame(self.root, text="Output", padx=10, pady=10)
        self.output_frame.grid(row=1, column=0, padx=5, pady=5, sticky="we")

        self.solution_label = tk.Label(self.output_frame, text="Optimal Solution:")
        self.solution_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.solution_text = tk.Text(self.output_frame, width=30, height=3)
        self.solution_text.grid(row=0, column=1, padx=5, pady=5)

        self.objective_value_label = tk.Label(self.output_frame, text="Objective Function Value:")
        self.objective_value_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.objective_value_text = tk.Text(self.output_frame, width=30, height=1)
        self.objective_value_text.grid(row=1, column=1, padx=5, pady=5)

    def update_variables_constraints(self):
        num_variables = int(self.num_variables_entry.get())
        num_constraints = int(self.num_constraints_entry.get())

        # Clear previous input widgets
        for widget in self.input_frame.winfo_children():
            widget.destroy()

        self.input_variables = []
        self.input_constraints = []
        self.input_b_values = []

        self.num_variables_label = tk.Label(self.input_frame, text="Number of variables:")
        self.num_variables_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.num_variables_entry = tk.Entry(self.input_frame)
        self.num_variables_entry.grid(row=0, column=1, padx=5, pady=5)
        self.num_variables_entry.insert(tk.END, str(num_variables))

        self.num_constraints_label = tk.Label(self.input_frame, text="Number of constraints:")
        self.num_constraints_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.num_constraints_entry = tk.Entry(self.input_frame)
        self.num_constraints_entry.grid(row=1, column=1, padx=5, pady=5)
        self.num_constraints_entry.insert(tk.END, str(num_constraints))

        self.cost_label = tk.Label(self.input_frame, text="Objective Function Coefficients:")
        self.cost_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.input_cost_entry = [tk.Entry(self.input_frame) for _ in range(num_variables)]
        for i, entry in enumerate(self.input_cost_entry):
            entry.grid(row=2, column=i+1, padx=5, pady=5)
        self.input_variables.append(self.input_cost_entry)

        for i in range(num_constraints):
            label = tk.Label(self.input_frame, text=f"Constraint {i+1}:")
            label.grid(row=i+3, column=0, padx=5, pady=5, sticky="w")
            constraint_entry = [tk.Entry(self.input_frame) for _ in range(num_variables)]
            for j, entry in enumerate(constraint_entry):
                entry.grid(row=i+3, column=j+1, padx=5, pady=5)
            self.input_constraints.append(constraint_entry)
            b_label = tk.Label(self.input_frame, text="B Value:")
            b_label.grid(row=i+3, column=num_variables+1, padx=5, pady=5, sticky="w")
            b_entry = tk.Entry(self.input_frame)
            b_entry.grid(row=i+3, column=num_variables+2, padx=5, pady=5)
            self.input_b_values.append(b_entry)

        self.solve_button = tk.Button(self.root, text="Solve", command=self.solve_linear_program)
        self.solve_button.grid(row=2, column=0, padx=5, pady=5)

    def solve_linear_program(self):
        num_variables = int(self.num_variables_entry.get())
        num_constraints = int(self.num_constraints_entry.get())

        cost = [float(entry.get()) for entry in self.input_variables[0]]
        A = []
        b = []

        for i in range(num_constraints):
            constraint = [float(entry.get()) for entry in self.input_constraints[i]]
            A.append(constraint)
            b.append(float(self.input_b_values[i].get()))

        result = self.solve_lp(cost, A, b)

        solution_str = "\n".join([f"Variable {var+1}: {val}" for var, val in result[1]])
        self.solution_text.delete("1.0", tk.END)
        self.solution_text.insert(tk.END, solution_str)

        self.objective_value_text.delete("1.0", tk.END)
        self.objective_value_text.insert(tk.END, str(result[2]))

    def solve_lp(self, cost, A, b):
        tableau = self.initial_tableau(cost, A, b)

        while self.can_improve(tableau):
            pivot = self.find_pivot_index(tableau)
            self.pivot_about(tableau, pivot)

        return tableau, self.primal_solution(tableau), self.objective_value(tableau)

    def initial_tableau(self, c, A, b):
        tableau = [row[:] + [x] for row, x in zip(A, b)]
        tableau.append([ci for ci in c] + [0])
        return tableau

    def primal_solution(self, tableau):
        columns = self.transpose(tableau)
        indices = [j for j, col in enumerate(columns[:-1]) if self.is_pivot_col(col)]
        return [(colIndex, self.variable_value_for_pivot_column(tableau, columns[colIndex]))
                for colIndex in indices]

    def objective_value(self, tableau):
        return -(tableau[-1][-1])

    def can_improve(self, tableau):
        lastRow = tableau[-1]
        return any(x > 0 for x in lastRow[:-1])

    def find_pivot_index(self, tableau):
        column_choices = [(i, x) for (i, x) in enumerate(tableau[-1][:-1]) if x > 0]
        column = min(column_choices, key=lambda a: a[1])[0]

        if all(row[column] <= 0 for row in tableau):
            raise Exception('Linear program is unbounded.')

        quotients = [(i, r[-1] / r[column])
                     for i, r in enumerate(tableau[:-1]) if r[column] > 0]

        if self.more_than_one_min(quotients):
            raise Exception('Linear program is degenerate.')

        row = min(quotients, key=lambda x: x[1])[0]

        return row, column

    def pivot_about(self, tableau, pivot):
        i, j = pivot

        pivotDenom = tableau[i][j]
        tableau[i] = [x / pivotDenom for x in tableau[i]]

        for k, row in enumerate(tableau):
            if k != i:
                pivotRowMultiple = [y * tableau[k][j] for y in tableau[i]]
                tableau[k] = [x - y for x, y in zip(tableau[k], pivotRowMultiple)]

    def is_pivot_col(self, col):
        return (len([c for c in col if c == 0]) == len(col) - 1) and sum(col) == 1

    def variable_value_for_pivot_column(self, tableau, column):
        pivotRow = [i for (i, x) in enumerate(column) if x == 1][0]
        return tableau[pivotRow][-1]

    def transpose(self, A):
        return [self.column(A, j) for j in range(len(A[0]))]

    def column(self, A, j):
        return [row[j] for row in A]

    def more_than_one_min(self, L):
        if len(L) <= 1:
            return False

        x, y = heapq.nsmallest(2, L, key=lambda x: x[1])
        return x == y

root = tk.Tk()
app = LinearProgramSolver(root)
root.mainloop()